# Validation Notebook

## Import Library

In [1]:
import warnings
import pickle
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, ConvLSTM2D, BatchNormalization
import tensorflow as tf
import keras

## Import Satellite Data for Unseen Locations
#### Define Functions Used to Import Data

In [2]:
# Opens pickled data from scraped files, which contains vv and vh data for unseen locations
def read_file(month):
    with open(f'{month}.pkl', 'rb') as file:
        return pickle.load(file)

In [3]:
# Process datasets to be uniform in the number of timsteps and pixels per sample 
def reshapeData3D(dataset, pixel, instances):
    dataset_reshaped1 = []
    for i in dataset:
        if i.shape[2] > pixel and i.shape[1] > pixel:
            dataset_reshaped1.append(i[:,:(pixel-i.shape[1]),:(pixel-i.shape[2])])
        elif i.shape[1] > pixel:
            dataset_reshaped1.append(i[:,:(pixel-i.shape[1]),:])
        elif i.shape[2] > pixel:
            dataset_reshaped1.append(i[:,:,:(pixel-i.shape[2])])
        else:
            dataset_reshaped1.append(i)

    dataset_reshaped = []
    for i in dataset_reshaped1:
        if i.shape[0] >instances:
            dataset_reshaped.append(i[:instances,:,:])
        else:
            dataset_reshaped.append((i[:,:,:]))
    return [np.transpose(i) for i in dataset_reshaped]

#### Import Data

In [4]:
#Standardising the shape of VV and VH data samples
VH = reshapeData3D(read_file('APR-AUG_VH'), 90, 24)
VV = reshapeData3D(read_file('APR-AUG_VV'), 90, 24)

#Performing matrix calculations to derive the RGB values for each sample
RGB = []
for i in range(len(VV)):
    RGB.append(np.reshape(np.stack([VH[i], VV[i], np.divide(VH[i], VV[i])]), (1,24, 90, 90, 3)))
    
#Reshaping training data to represent the tuple: (number of samples, timesteps, dimensions, pixels X, pixels Y)
#the last 250 rows are VV and VH data for unseen locations
X_test = np.array(RGB).reshape(850, 24, 3, 90, 90)[600:] 

In [5]:
X_test.shape

(250, 24, 3, 90, 90)

## Import Trained Model

In [6]:
# Open pickle file
with open('Model (1).pkl', 'rb') as file:
    # Load trained model from pickle file
    model = pickle.load(file)

2023-04-10 14:21:37.537842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Show the summary of trained model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 24, 3, 90, 32)     140672    
                                                                 
 batch_normalization (BatchN  (None, 24, 3, 90, 32)    1080      
 ormalization)                                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 24, 3, 90, 64)     221440    
                                                                 
 dropout (Dropout)           (None, 24, 3, 90, 64)     0         
                                                                 
 batch_normalization_1 (Batc  (None, 24, 3, 90, 64)    1080      
 hNormalization)                                                 
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 24, 3, 90, 128)    8

In [8]:
## Used Trained Model to Predict Unseen Locations

In [9]:
y_pred = model.predict(X_test)


result = []
for i in y_pred:
    # the output layer use sigmoid function, 
    # if the output is > 0.5, we predict 'Rice'
    # otherwise, we predict 'Non Rice'
    if i>0.5:
        result.append('Rice')
    else:
        result.append('Non Rice')

8/8 [==============================] - 23s 3s/step


In [10]:
submission = pd.read_csv('challenge_1_submission_template_correct_columns_fixed.csv')
# store the prediction in the submission
submission['target'] = result

In [11]:
submission

,id,target
0,"(10.18019073690894, 105.32022315786804)",Rice
1,"(10.561107033461816, 105.12772097986661)",Rice
2,"(10.623790611954897, 105.13771401411867)",Rice
3,"(10.583364246115156, 105.23946127195805)",Non Rice
4,"(10.20744446668854, 105.26844107128906)",Rice
...,...,...
245,"(10.308283266873062, 105.50872812216863)",Non Rice
246,"(10.582910017285496, 105.23991550078767)",Non Rice
247,"(10.581547330796518, 105.23991550078767)",Non Rice
248,"(10.629241357910818, 105.15315779432643)",Rice
